In [ ]:
import os
import pickle
import mediapipe as mp
import cv2


In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.3)

In [ ]:
DATA_DIR = "F:/ISL/data"

In [ ]:
data = []
labels = []

for dir_ in os.listdir(DATA_DIR):
    dir_path = os.path.join(DATA_DIR, dir_)

    if not os.path.isdir(dir_path):
        continue

    for img_path in os.listdir(dir_path):
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        print(f"Processing: {img_path}")
        img_path_full = os.path.join(dir_path, img_path)
        img = cv2.imread(img_path_full)

        if img is None:
            print(f"[Warning] Failed to load image: {img_path_full}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            hand1 = []
            hand2 = []

            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                hand_data = []
                x_ = [lm.x for lm in hand_landmarks.landmark]
                y_ = [lm.y for lm in hand_landmarks.landmark]

                width = max(x_) - min(x_)
                height = max(y_) - min(y_)
                if width == 0 or height == 0:
                    continue  # Avoid division by zero

                for lm in hand_landmarks.landmark:
                    norm_x = (lm.x - min(x_)) / width
                    norm_y = (lm.y - min(y_)) / height
                    hand_data.append(norm_x)
                    hand_data.append(norm_y)

                if idx == 0:
                    hand1 = hand_data
                elif idx == 1:
                    hand2 = hand_data

            if not hand1:
                hand1 = [0.0] * 42
            if not hand2:
                hand2 = [0.0] * 42

            data_aux = hand1 + hand2
            data.append(data_aux)
            labels.append(dir_)
        else:
            print(f"Skipping sample {img_path}: no hand detected.")


In [ ]:
# Save the collected data
with open('data.pickle', 'wb') as f:
    pickle.dump((data, labels), f)

print("Data collection complete. Saved to data.pickle.")
